In [1]:
#start
import gspread #type:ignore NOQA
import imaplib
import email
import json

#connecting to GSeets and selecting the file
sa = gspread.service_account(filename="sa_creds.json")

#read credentials from json
with open("ym_creds.json", "r") as f:
    jmCreds = json.load(f)

groceryBrandDoneEmails = [] #list to write to GSheets
def groceryBrandEmails(GroceryBrand: str, emailList):
    for i in emailList:
        f = [GroceryBrand]
        f.append(i)
        groceryBrandDoneEmails.append(f)

sh = sa.open("Grocery")
#selecting sheets:
wksEmails = sh.worksheet("Emails")
wksSummaryMaxima = sh.worksheet("MaximaSummarized")

#getting data from wksEmails sheet
# wksEmails.get()
maximaAlreadyIn = [i[1] for i in wksEmails.get() if 'Maxima' in i[0]]

In [2]:
# Connect to the Yahoo Mail IMAP server
imap_server = imaplib.IMAP4_SSL("imap.mail.yahoo.com")
# Login to the account
imap_server.login(jmCreds["email"], jmCreds["password"])
# Select the "Inbox" folder
imap_server.select("Inbox")

# Search for all emails: status, email_ids = imap_server.search(None, "ALL")
# Search for all emails from MAXIMA
status, email_ids = imap_server.search(None, "FROM noreply.code.provider@maxima.lt")
maximaEmails = email_ids[0].decode().split()

maximaToAdd = [i for i in maximaEmails if i not in maximaAlreadyIn]

In [3]:
maximaToAdd

['5468', '5482', '5495', '5497', '5529']

In [4]:
receiptSummaryDataFULL = []
receiptSummaryData = []
def readMaximaReceiptSummary(EmailId: str, receipt):
    receiptSummaryData.clear()
    for i in range(0, 11):
        receiptSummaryData.append('')
    receiptSummaryData[0] = EmailId
    for i in receipt:
        #Receipt_id
        if '<pre>Kvitas ' in i:
            receiptSummaryData[1] = i.split()[1]
        #BankReceipt_id
        if 'KVITO NR' in i:
            receiptSummaryData[2] = i.split()[2]
        #Receipt_Document#
        if 'DOKUMENTO NR' in i:
            receiptSummaryData[3] = i.split()[2]
        #RRN
        if 'RRN:' in i:
            receiptSummaryData[4] = i.split()[1]
        #Date
        if 'LTF NM ' in i:
            receiptSummaryData[5] = f'{i.split()[3]}-{i.split()[4]}-{i.split()[5]}'
        #Time
        if 'LTF NM ' in i:
            receiptSummaryData[6] = i.split()[6]
        #GroceryBrand
        if 'MAXIMA LT' in i:
            receiptSummaryData[7] = 'MAXIMA LT'
        #Address
        if 'MAXIMA LT, UAB' in i:
            receiptSummaryData[8] = i.split('<br />')[1].split(' Kasa Nr.')[0][:-1]
        #TotalAmount
        if 'Apsipirkimo suma:' in i:
            receiptSummaryData[9] = i.split(': ')[1][:-4]
        #TotalDiscount
        if 'Kvito nuolaidų suma:' in i:
            receiptSummaryData[10] = i.split(': ')[1][:-4]
    receiptSummaryDataFULL.append(list(receiptSummaryData))


In [5]:
for i in maximaToAdd:
    status, email_data = imap_server.fetch(i, "(RFC822)")
    email_message = email.message_from_bytes(email_data[0][1])
    
    msg1 = email_message.get_payload()[0]
    msg1body = msg1.get_payload(decode=True)
    singleEmail = msg1body.decode().split('\r\n')
    readMaximaReceiptSummary(i, singleEmail)
wksSummaryMaxima.append_rows(receiptSummaryDataFULL)

{'spreadsheetId': '10-mjgW6dVIMLh5iVe6NbyIbI_N3XZ0mOFGlxilp4W0s',
 'tableRange': 'MaximaSummarized!A1:K1',
 'updates': {'spreadsheetId': '10-mjgW6dVIMLh5iVe6NbyIbI_N3XZ0mOFGlxilp4W0s',
  'updatedRange': 'MaximaSummarized!A2:K6',
  'updatedRows': 5,
  'updatedColumns': 11,
  'updatedCells': 55}}

In [6]:
# Disconnect from the server
imap_server.close()
imap_server.logout()

('BYE', [b'IMAP4rev1 Server logging out'])

In [6]:
receiptSummaryDataFULL

[['5468',
  '219/2658',
  '458107',
  '446176-221219195735',
  '235386564590',
  '2022-12-19',
  '19:57:49',
  'MAXIMA LT',
  'Ukmergės g. 369, Vilnius',
  '23.89 EUR',
  '9.30 EUR'],
 ['5482',
  '165/1352',
  '422078',
  '552388-221223164617',
  '235794552559',
  '2022-12-23',
  '16:46:35',
  'MAXIMA LT',
  'Ozo g. 25 Vilnius',
  '60.54 EUR',
  '8.24 EUR'],
 ['5495',
  '193/2199',
  '781175',
  '199908-221228201743',
  '236211842171',
  '2022-12-28',
  '20:17:59',
  'MAXIMA LT',
  'Verkių g.31C, Vilnius',
  '6.08 EUR',
  '0.59 EUR'],
 ['5497',
  '250/1104',
  '190882',
  '840235-221229202323',
  '236313658535',
  '2022-12-29',
  '20:23:39',
  'MAXIMA LT',
  'Ukmergės g. 369, Vilnius',
  '13.48 EUR',
  '3.08 EUR'],
 ['5529',
  '207/1826',
  '483213',
  '340496-230109193907',
  '300930628307',
  '2023-01-09',
  '19:39:25',
  'MAXIMA LT',
  'Ozo g. 18, Vilnius',
  '29.22 EUR',
  '0.13 EUR']]

In [4]:
for i in maximaToAdd:
    status, email_data = imap_server.fetch(i, "(RFC822)")
    email_message = email.message_from_bytes(email_data[0][1])

In [5]:
#get message bu ID
status, email_data = imap_server.fetch("5482", "(RFC822)")
email_message = email.message_from_bytes(email_data[0][1])

In [6]:
msg1 = email_message.get_payload()[0]
msg1body = msg1.get_payload(decode=True)
msg2 = email_message.get_payload()[0]
msg2body = msg2.get_payload(decode=True)
msg3 = email_message.get_payload()[0]
msg3body = msg3.get_payload(decode=True)

In [7]:
# print(msg1.get_payload(decode=True), msg2.get_payload(decode=True))
print(msg2body.decode())

<html><body>


<html>

<body>

    <div class="previewText" style="display:none; max-height: 0px; overflow: hidden">
            <p>Apsipirkimo suma: 60.54 EUR</p>
            <br />
                <p>Kvito nuolaidų suma: 8.24 EUR</p>
                <br />
                <p> MAXIMOS PINIGŲ likutis: 0.59 EUR </p>
            <p>
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;

In [92]:
x = msg2body.decode().split('\r\n')
for e, i in enumerate(x):
    print(e, x[e])
# x[10]

0 <html><body>
1 
2 
3 <html>
4 
5 <body>
6 
7     <div class="previewText" style="display:none; max-height: 0px; overflow: hidden">
8             <p>Apsipirkimo suma: 60.54 EUR</p>
9             <br />
10                 <p>Kvito nuolaidų suma: 8.24 EUR</p>
11                 <br />
12                 <p> MAXIMOS PINIGŲ likutis: 0.59 EUR </p>
13             <p>
14                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
15                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
16                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
17                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwn

In [53]:
# receiptSummaryData = [['','','','','','','','','','','']]
readMaximaReceiptSummary('1234' ,x)
# receiptSummaryData

In [54]:
wksSummaryMaxima.append_rows(receiptSummaryData)
# receiptSummaryData

{'spreadsheetId': '10-mjgW6dVIMLh5iVe6NbyIbI_N3XZ0mOFGlxilp4W0s',
 'tableRange': 'MaximaSummarized!A1:K2',
 'updates': {'spreadsheetId': '10-mjgW6dVIMLh5iVe6NbyIbI_N3XZ0mOFGlxilp4W0s',
  'updatedRange': 'MaximaSummarized!A3:K3',
  'updatedRows': 1,
  'updatedColumns': 11,
  'updatedCells': 11}}

In [9]:
for i in x:
    if 'MAXIMA LT, UAB' in i:
        print(i.split('<br />')[1].split(' Kasa Nr.')[0][:-1])

NameError: name 'x' is not defined